In [175]:
from sklearn.metrics import accuracy_score
import numpy as np
import tensorflow as tf
import json
import scipy.io
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import cv2
import h5py
import scipy.io

plt.rcParams["figure.figsize"] = 8,8

In [176]:
from scipy.signal import find_peaks
def spike_finder(traces, max_t=0.4, distance=3):
    spikes_all = []
    for trace in traces:
        maximum = np.max(np.array(trace))
        min_height = max_t * maximum
        spikes = find_peaks(x=trace, height=min_height, distance=distance)
        spikes_all.append(spikes[0].tolist())
        
    return spikes_all

In [177]:
def load_ground_truth_generation_data(path, gui=False):
    """
    Loads ground truth data generated by MATLAB annotation tool.

    Returns:
        filters_binary 
        600 x 500 x 500 bool

        events_in_frames 
        11998 x {eventsInThisFrame} int

        events_for_filters 
        600 x {eventsForThisFilter} intfor i in range(contours_all.shape[0]):

    coord = []
    for j in range(len(contours_all[i][0])):

        y = contours_all[i][0][j][0][1]
        x = contours_all[i][0][j][0][0]
        coord.append([x,y])

    coord.append(coord[0])
    coords_all_pca.append(coord)

    xs, ys = zip(*coord)
    plt.plot(xs,ys, c=plt.get_cmap('autumn')(i/contours_all.shape[0]))
fig = plt.gcf()
plt.show()

        filter_centroids
        600 x 2 int
    """
    ground_truth_generation_data = scipy.io.loadmat(path)

    # 600 x 500 x 500 bool
    filters_binary = ground_truth_generation_data["filtersBinary"]
    filters_binary = np.array(
        [
            np.array(filters_binary[:, :, i]).transpose()
            for i in range(np.shape(filters_binary)[2])
        ],
        dtype=np.bool,
    )

    # 11998 x {eventsInThisFrame} int
    events_in_frames = ground_truth_generation_data["eventsInFrames"]

    if not gui:
        events_in_frames = np.array(
            [events_in_frames[0, i][0] - 1 for i in range(np.shape(events_in_frames)[1])]
        )

    else:
        
        events_in_frames_new = []
        for event in events_in_frames[0]:
            if np.array(event).size == 0:
                events_in_frames_new.append(np.array([]))
            else:
            
                events_in_frames_new.append(np.array(event[0]))
        events_in_frames = np.array(events_in_frames_new)
        
        
    # 600 x {eventsForThisFilter} int
    events_for_filters = ground_truth_generation_data["eventsForFilters"]

    if not gui:
        events_for_filters = np.array(
            [
                events_for_filters[0, i][0] - 1
                for i in range(np.shape(events_for_filters)[1])
            ]
        )
    else:
        events_for_filters_new = []
        for event in events_for_filters[0]:
            if np.array(event).size == 0:
                events_for_filters_new.append(np.array([]))
            else:
            
                events_for_filters_new.append(np.array(event[0]))
        print(events_for_filters_new[0][0])
        events_for_filters = np.array(events_for_filters_new)

    # 600 x 2 int
    filter_centroids = ground_truth_generation_data["filterCentroids"]
    filter_centroids = [[y, x] for [x, y] in filter_centroids]
    return [filters_binary, events_for_filters, filter_centroids, events_in_frames]

In [178]:
def find_centroids_from_binary_mask(mask):
    
    #float_img = float(mask)
    img = np.array(mask * 255, dtype = np.uint8)



    # convert the grayscale image to binary image
    ret,thresh = cv2.threshold(img,127,255,0)

    # calculate moments of binary image
    M = cv2.moments(thresh)

    # calculate x,y coordinate of center
    cX = int(M["m10"] / M["m00"])
    cY = int(M["m01"] / M["m00"])
    
    return cX, cY


In [179]:
def binary_mask_from_contour(coords):
    
    width = 500
    height = 500
    
    polygon = [tuple(x) for x in coords]

    img = Image.new('L', (width, height), 0)
    ImageDraw.Draw(img).polygon(polygon, outline=1, fill=1)
    mask = np.array(img)
    return mask

In [180]:
def load_preprocessed_movie(movie_path):
    movie = h5py.File(movie_path, 'r')
    movie = movie['1']
    return movie

In [210]:
#we wanna look like this
pca_ica_w = scipy.io.loadmat("/home/user/Documents/single_session/dataPackedForGeneration.mat")
eventsForFilters = pca_ica_w["eventsForFilters"]
eventsInFrames = pca_ica_w["eventsInFrames"]
filterCentroids = pca_ica_w["filterCentroids"]
filtersBinary = pca_ica_w['filtersBinary']

# #load gui mat
#gui_mat = scipy.io.loadmat('/home/user/Documents/single_session/GUI/synthetic/gui_extracted_wo_added.mat')
gui_mat = scipy.io.loadmat('/home/user/Documents/single_session/GUI/singleshot/gui_extracted.mat')
coords_gui = gui_mat["coordinates"][0]
events = gui_mat['events'][0]
traces = gui_mat['traces']
#load movie belonging to gui data
#movie_path = '/home/user/Documents/synthetic_datasets/sequence_9/video/movie.h5'
movie_path = '/home/user/Documents/single_session/preprocessed/preprocessedMovie.h5'
movie = load_preprocessed_movie(movie_path)







# events_cnmfe_added = spike_finder(traces[340:len(traces)])
# events_cnmfe= np.array([np.array(x) for x in events_cnmfe_added])

    

# events_new = []
# for event in events[0:340]:
#     events_new.append(event)
# for event in events_cnmfe:
#     events_new.append([event])
# events_new = np.array(events_new)

events_new = events
# filters binary
masks_gui = []
for coord in coords_gui:
    mask = binary_mask_from_contour(coord.tolist())
    masks_gui.append(mask)

masks_gui = np.array(masks_gui)
filtersBinaryGUI = np.transpose(masks_gui, (1, 2, 0))



# eventsForFiltersGUI
eventsForFiltersGUI = []
for i, event in enumerate(events_new):
    if not list(event):
        print(i)
        eventsForFiltersGUI.append(np.array([[]]))
    else:
        eventsForFiltersGUI.append(np.array(event[0]))

# filter centroids
filterCentroidsGUI = []
for mask in masks_gui:
    cX, cY = find_centroids_from_binary_mask(mask)
    filterCentroidsGUI.append([cX,cY])
filterCentroidsGUI = np.array(filterCentroidsGUI, dtype="uint16")

241
249
265
270
272
277
281
289
293
294
306
307
315
320
321
322
323
330
338
339


In [182]:
eventsForFiltersGUI[0]

array([ 565, 1002, 1663, 3223, 3283, 4735, 6104, 6216, 6224, 8315, 8640],
      dtype=uint16)

In [220]:
#we wanna look like this
pca_ica_w = scipy.io.loadmat("/home/user/Documents/single_session/dataPackedForGeneration.mat")
eventsForFilters = pca_ica_w["eventsForFilters"]
eventsInFrames = pca_ica_w["eventsInFrames"]
filterCentroids = pca_ica_w["filterCentroids"]
filtersBinary = pca_ica_w['filtersBinary']

# #load gui mat
# gui_mat = scipy.io.loadmat('/home/user/Documents/single_session/GUI/synthetic/gui_extracted_wo_added.mat')
# #gui_mat = scipy.io.loadmat('/home/user/Documents/single_session/GUI/singleshot/gui_extracted.mat')
# coords_gui = gui_mat["coordinates"][0]
# events = gui_mat['events'][0]
# traces = gui_mat['traces']
# #load movie belonging to gui data
# movie_path = '/home/user/Documents/synthetic_datasets/sequence_9/video/movie.h5'
# #movie_path = '/home/user/Documents/single_session/preprocessed/preprocessedMovie.h5'
# movie = load_preprocessed_movie(movie_path)


# Opening JSON file
f = open('/home/user/Documents/synthetic_datasets/sequence_9/data.json',)
  
# returns JSON object as 
# a dictionary
data = json.load(f)
coords_gui = data['neuron_coordinates']
events = data['neuron_events']
traces = data['neuron_activities']
# Closing file
f.close()




# events_cnmfe_added = spike_finder(traces[340:len(traces)])
# events_cnmfe= np.array([np.array(x) for x in events_cnmfe_added])

    

# events_new = []
# for event in events[0:340]:
#     events_new.append(event)
# for event in events_cnmfe:
#     events_new.append([event])
# events_new = np.array(events_new)

events_new = events
# filters binary
masks_gui = []
for coord in coords_gui:
    mask = binary_mask_from_contour(coord)
    masks_gui.append(mask)

masks_gui = np.array(masks_gui)
filtersBinaryGUI = np.transpose(masks_gui, (1, 2, 0))



# eventsForFiltersGUI
eventsForFiltersGUI = []
for event in events_new:
    if not list(event):
        print("yes")
        eventsForFiltersGUI.append(np.array([[]]))
    else:
        eventsForFiltersGUI.append(np.array(event))

# filter centroids
filterCentroidsGUI = []
for mask in masks_gui:
    cX, cY = find_centroids_from_binary_mask(mask)
    filterCentroidsGUI.append([cX,cY])
filterCentroidsGUI = np.array(filterCentroidsGUI, dtype="uint16")


In [212]:
filterCentroidsGUI.shape

[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]


In [183]:
i = 0
size = 0
for event in events_new:
    size += np.array(event).size
    if np.array(event).size == 0:
        i += 1
print(i)
print(size)

20
88434


In [213]:

eventsInFramesGUI = []
for i in range(movie.shape[0]):
    frame = np.zeros((500,500))
    frame_events = []
    for j, (events_single) in enumerate( events_new):
        if not list(events_single):
            continue
        for event in events_single[0]:
            if event == i:
                frame_events.append(j)
                                              
    
    
    if not frame_events:
        #frame_events = np.expand_dims(np.array([],  dtype='float64'), axis=1)
        frame_events = np.array([],  dtype='float64')
    else:
        frame_events = np.array(frame_events, dtype='uint16')
        print(frame_events)
    eventsInFramesGUI.append(frame_events)
    

eventsInFramesGUI = np.array(eventsInFramesGUI, dtype='object')


[ 61  92 205 387 489 503 592]
[224 388 429 476]
[376 456 503 506 520 528 562]
[402 444 460 488 499]
[352 378 438 471 487 580 607]
[ 83 165 250 316 485 532 550 568]
[206 354 480 537 615]
[333 444 521 569 575 614]
[ 23 176 438 555]
[254 351 381 387 392 414 458 514 524 572]
[121 301 347 365 371 383 391 437 466 485 499 536 564 593]
[262 352 396 427 477 532]
[464 488 498 502 562 592]
[  6  79 189 411 429 460 480 518 555 598]
[136 461 552]
[ 31 352 464 488 565 576]
[ 63 100 195 237 238 319 425 438 479 552]
[ 46 481 562 572 581]
[346 532 543]
[ 84 110 383 405 417 531 552 572]
[189 396 425 478]
[258 342 351]
[ 55 383 392 396 425 508 581]
[487 532 614 615]
[ 43 198 237 344 436 456 488 521 531 565 566]
[ 22  32 497 504 536]
[ 16 210 251 258 351 352 409 537 551 552 562 593]
[100 238 319 377 481 507 572]
[  6 598 614]
[346 535 615]
[ 84  92 392 396 499 537 581 614]
[ 43 177 354 405 460 497 565 576]
[ 23  32 161 319 342 392 395 464 474 532 537 538 575]
[146 258 371 373 383 425 427 552 555]
[141 564

[371 428 457 493 521 560 603]
[376 377 429 455 503 525 546 551 581]
[ 61 275 373 381 392 414 458 460 486 497 568]
[225 351 368 457 461 499 566 597]
[ 53  85 408 472 523 532]
[ 30  92 126 368 402 537 557 580 584]
[198 373 553 555 590 598]
[127 313 571 576]
[ 57 258 491 501 540 573]
[ 38  70 472 557]
[478 493 496 523 528]
[113 180 301 358 364 370 440 602]
[ 61 378 381 392 414 437 521 575 582]
[  7  33  48  58 359 457 491]
[ 77 381 392 408 414 435 437 503 555]
[267 369 444 482 499 524 569 585 594]
[364 479 486 540 581]
[ 64 127 206 393 494 505 559 595]
[106 119 161 216 236 359 366 368 401 415 455 603]
[258 409 437 447 482 524 568 578]
[245 472 503 536 539]
[ 84  97 298 372 413 471 532 538 552 576]
[ 92 164 220 260 352 400 402 465 497 615]
[  6 354 378 407 521 608 614]
[254 371]
[  9  33  63 177 267 376 381 414 472 539 555]
[205 225 373 392 477 553 566 569]
[206 440 491 557]
[127 305 415 528 575]
[106 358 385 493 501 571 586]
[  7 147 159 408 436 476]
[ 61 136 258 387 437 474 584]
[ 27 374

[314 396 475 562]
[393 406 450 500 505 508 532 590 611]
[ 72 228 258 366 367 425 428 447 521 543 553 559 577 595]
[427 438 443 541 564]
[507]
[ 64 377 393 408 411 447 479 481 494 521 579]
[ 49 109 194 303 347 397 500 543]
[363 407 480 499 503 514 555]
[374 482 611]
[186 396 401 438 440 506 509 521 532 553 574]
[359 400 447 475]
[119 206 279 357 366 406 411 430 443 481 494 543 566]
[ 28  39 369 393 427 500 507 593]
[179 196 208 251 359 374 384 397 411 442]
[ 14  61 311 420 455 499 503 526 536 573]
[235 363 396 402 508 514]
[106 198 347 369 371 456 457 482 511 515]
[105 161 379 400 411 494]
[ 35 185 217 447 468 521]
[ 57 377 403 481 615]
[ 64 107 136 224 314 502 578]
[109 134 164 177 235 369 383 398 499 506 564]
[ 61 372 415 420 503 510 538 566]
[ 45  70 206 407 456 473 504 537 568]
[119 251 259 359 396 442 461 532 574 580 592 604]
[220 317 383 408 480 513 514]
[ 41  54  92 198 258 434 438 491 588]
[346 378 471 499]
[286 351 352 438 478 481 497 547 615]
[109 193 207 316 354 378 469 565 5

[ 43  56  94 332 354 474 477 614 615]
[ 92 301 381 392 414]
[212 382 400 406 413 425 485 504 513 537 594]
[193 225 344 371 372 385 399 447 517 604]
[ 27 147 382 396 431 479 586 600]
[ 84 132 245 302 342 369 373 415 514 530]
[397 431 468 471 546 580 603]
[ 38 383]
[305 388 454 461 474 476 532 533 538 540 590]
[ 54 135 150 225 316 384 566 574]
[146 378 500 544 553 575]
[480]
[ 97 407 438 442 465 478 526 548 549 572 587 615]
[468 481 487 499 564]
[167 280 482 551 613 614 615]
[ 11  18 131 354 377 460]
[ 81 258 493 565 581 613]
[ 43  69 377 547]
[165 254 311 378 381 392 414 468 593 601 615]
[233 301 376 429 540 586 614]
[ 55  64 214 237 387]
[177 256 373 374 382 396 462 491 495 511 533 537 578 580]
[145 454 458 485 504 527]
[225 231 403 413 592]
[ 41  54 193 376 383 384 406 426 457 461 486 514 573]
[391 397 424 549 584]
[ 85 317 379 454 463 560]
[ 86 247 275 362 364 377 407]
[299 392 498 600]
[ 27 374 385 419 431 472 501 543 578 603]
[160 216 220 337 358 399 452 473 478 491 507 508 528]
[ 

[ 75 125 205 364 428 457 472]
[ 85 145 371 377 495 536 539 564]
[143 147 170 399 428 440 493 496 501 519 602]
[ 38  70 159 303 473 478 503 553]
[482 499 537]
[ 33 538 544]
[ 56 377 381 392 414 480]
[ 68 200 437 476]
[ 54 145 373 430 486 497 536]
[151 258 267 359 363 388 400 433 505 548 585]
[132 255 299 319 524 527 592 599]
[105 119 475 479 525 540]
[406 428 500 523 541 595]
[169 228 369 417 427 440 447 494 507 521]
[ 57 164 217 393 482 540 571 585]
[112 125 438 443 468 499]
[ 49 149 240 377 425 440 447 508 562 580]
[368 397 402 539 541 581 604]
[ 39  64 208 303 506]
[ 28  56  99 109 313 314 326 420 468 503 521 564]
[258 357 372 455 538 568]
[ 54  97 251 298 347 373 407 413 437 480 491 566]
[376 457 510 588 608]
[ 41 407 437 444 455 461 471 506 532 541]
[ 40 190 206 317 440 466]
[149 258 373 396 400 456 500 514]
[382 385 406 415 477 495 511]
[ 54 299 328 468 474 485 493 497 530 547 548 574]
[ 79 373 378 403 471 480 537]
[395 466 473 481]
[ 43  59 207 216 316 482 517 566]
[111 114 354 3

...

[ 43  74  79 110 232 418 459 555 560]
[ 41 417 430 492 566 572]
[ 68 428 469 481 565]
[  3  81 108 311 346 516]
[ 84 142 189 308 497]
[ 52 210 351 480 492 552]
[203 386 417 443 482 506 516 535 541]
[ 18  76 112 122 171 375 434 446 589]
[239 352 365 388 392 416 420 440 444 457 484 575]
[ 63 163 219 231 290 340 454 461 470 504 594]
[ 29 178 444]
[ 41 182 350 356 398 480]
[312 334 421 507 508 546 552 579]
[488 526 573 597]
[ 60 377 438 466]
[ 68 144 197 373 422 455 480 543 560 592]
[ 35 193 261 418 467 487]
[212 238 268 397 399 494 525 531 555 601]
[  7  41  43  84 173 266 269 526]
[111 437 443 459 509 551 565]
[ 74 351 405 428 469 559 576 598]
[531 535 552]
[494 499 515 565 598 604]
[134 174 206 384 481 487 569 572 578 579 610]
[ 68 125 210 250 386 405 441 552 581]
[233 379 416 451 484 529]
[353 436 565 566 583]
[ 52  81 108 138 189 346 438 481 492 543 572]
[193 231 439 441 469 551]
[ 70 368 377 378 424 428 526 533 536 546]
[ 45 326 353 457 518 535 544]
[ 22 236 437 492 521 552]
[134 192

[119 308 374 496 506 517 536 563]
[178 375 381 392 414 431 439 455 484 503 582]
[247 355 372 420 460 517 540 588]
[ 73 125 191 398 475 525 534]
[ 18 357 377 381 392 414 421 439 453 470]
[149 267 367 446 506 570]
[121 373 402 437 477 482 533 557]
[181 367 372 421 518 520]
[ 62 141 283 364 375 431 448 493 496]
[106 178 222 370 385 391 399 419 428 477 478 482 492 501 528 579]
[436 542]
[147 199 384 399 416 428 431 439 453 482 494 519 520 530 543 556 581 585]
[119 212 221 269 358 368 421 473 521 529 559]
[ 37 143 391 435 439 542]
[121 236 355 381 392 412 414 420 437 462 490 563]
[127 232 410 438 493 534 557]
[ 33 178 368 385 398 404 412 475 478 523]
[ 72 452 508 543 585]
[366 370 376 385 412 493 501 557]
[358 399 428 432 526]
[147 172 212 303 351 364 368 374 391 419 491 579 588]
[106 200 381 388 392 414 439 460 496 539 557]
[ 56 127 396 398 406 412 425 452 486 503 516]
[ 31  78 105 342 360 370 449 463 479 544 569]
[148 223 243 261 364 366 406 437 451 500]
[ 14 216 278 367 376 404 436 521 5

[207 219 385 424 519 571]
[ 25 134 260 364 374 423 452 500 601]
[166 210 371 477 526 579]
[103 182 198 370 577]
[191 269 360 462]
[308 364 370 385 399 416 437 540]
[  5  86 212 371 428 478 496 520]
[370 398 447 452 516 541]
[412 585]
[374 396 406 425 437 443 449 480 485 499 509 564 571 590]
[115 305 361 397 479 500 507 521 579]
[ 11  36 128 216 342 366 367 443 484 508 544]
[55]
[258 347 425 427 494 500 507 509 521 579]
[ 50 151 400 475 479 537]
[ 75 367 402 406 427 443 447 500 526 572 579 593]
[ 14 164 250 256 384 393 481]
[ 28  64 111 148 212 247 308 325 327 363 367 457 480 494 564]
[ 23  84 175 187 215 240 269 275 300 420 447 462]
[149 232 243 503 517 561]
[ 56 166 347 372 423 508 511 538 552 574]
[133 263 344 408 413 416 436 461 464 467 473 484 513 584]
[ 79 374 383 429 433 547 575]
[ 54 236 340 461 490 544 585]
[179 348 354 373 456 473]
[ 55 350 369 416 460 599]
[343 430 464 547]
[403 446 455 480 508 537]
[ 18  21 151 398 516 591]
[374 387 416 434 454]
[ 87 103 131 207 251 464 485 

[440 528 553 558 566 577]
[350 475 480]
[206 346 419 492 541 552 553 558 599]
[264 301 423 498 544]
[497 551]
[354 366 378 417 429 535 572]
[ 23 367 374 413 473 484 500 521 585]
[350 404 430 480 578]
[384 420 483 533 553 559]
[386 538 615]
[481 506 614]
[ 26  52 486 526 561]
[  6  46 420 497 525 527 583]
[210 585]
[ 34  55 347 417 452 462 552]
[  5   8  48 149 372 374 385 392 420 430 443 457 468 487 534 536 549]
[143 225 383 395 413 439 461 519 560]
[ 45  62 369 394 412 416 421 464 484 544 575]
[ 63  95 192 193 212 246 269 276 312 356 377 380 439 488]
[176 545]
[ 41 389 462]
[ 21 125 371 533]
[102 191 224 361 411 517 564]
[141 214 231 387 440 484 527 576 605]
[396 422 437 487 585]
[ 63 143 385 389 420 431 453 462 524 528 536 549 578]
[  8  21 225 238 436 461 490 534]
[18]
[525 561 585]
[173 419 478 533 566]
[ 52 258 391 423 524 535 569 582]
[  6  70 232 340 370 373 392 420 453 528 549 599]
[374 377 439]
[ 18 181 212 246 359 393 473 607]
[238 464 520 560 585]
[ 21 125 143 160 347 384 38

[ 77 229 283 352 358 374 394 439 495 501 557]
[ 19  21  89 147 354 383 385 410 469 493 500 560]
[ 78 431 445 534]
[164 254 395 404 415 471 484 492 519 563 577 587]
[378 412 453 456 556 559]
[161 165 173 194 369 473 524 532]
[345 509 559]
[267 378 381 392 414 420 533 582]
[  6  45 187 394 395 412 422 478 519]
[385 428 501 531 536 543]
[ 21  38 286 376 399 419 471 476 478 493 508]
[358 377 404 473 492 520 542]
[ 95 212 370 390 399 452 494 526 556 572]
[ 37  69 428 473 524]
[ 91 180 237 240 386 469 528]
[229 361 370 378 381 392 395 414 445 451 534 586]
[ 31  77 209 415 437 439 518]
[ 72 187 380 412 421 478 508 529]
[ 21 303 374 385 399 428 556 578]
[ 37 223 434 460 501 563]
[ 61  80  95 173 498 582]
[ 77 205 379 381 392 453 518 532]
[376 394 557]
[ 19 157 347 379 384 431 445 453 484 495 555]
[ 21  45  89 345 478 482 494 501 517]
[358 377 378 385 399 456 469 473 481 543]
[ 69  81 429 492 528 533]
[ 77 404 405 555 572 615]
[145 187 386 416 592]
[ 89 203 395 398 493 501 570]
[165 209 382 518

In [214]:
i = 0
size = 0
for event in eventsInFramesGUI:
    size += event.size
    if event.size == 0:
        i += 1

In [47]:
i

4

In [221]:
#save to matfile
mdic = {'eventsForFilters': eventsForFiltersGUI,
'eventsInFrames':  eventsInFramesGUI,
'filterCentroids': filterCentroidsGUI,
'filtersBinary': filtersBinaryGUI}
scipy.io.savemat("data_packed_for_generation_synth_GT.mat", mdic)

In [139]:
my_data = scipy.io.loadmat("data_packed_for_generation_synth.mat")
events = my_data['eventsForFilters']

In [140]:
events[0][0]

array([[628]])

In [222]:
[a1, a2, a3, a4] = load_ground_truth_generation_data("data_packed_for_generation_synth_GT.mat", gui=True)

336


/home/user/anaconda3/envs/hdf5/lib/python3.7/site-packages/ipykernel_launcher.py:62: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/home/user/anaconda3/envs/hdf5/lib/python3.7/site-packages/ipykernel_launcher.py:84: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [206]:
[b1, b2, b3, b4] = load_ground_truth_generation_data("/home/user/Documents/single_session/dataPackedForGeneration.mat", gui=False)

/home/user/anaconda3/envs/hdf5/lib/python3.7/site-packages/ipykernel_launcher.py:50: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/home/user/anaconda3/envs/hdf5/lib/python3.7/site-packages/ipykernel_launcher.py:72: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [229]:
a4

array([array([], dtype=float64),
       array([ 61,  92, 205, 387, 489, 503, 592], dtype=uint16),
       array([224, 388, 429, 476], dtype=uint16), ...,
       array([ 67, 410, 448, 525, 561, 563], dtype=uint16),
       array([380, 384, 385, 419, 431, 445, 498, 508], dtype=uint16),
       array([], dtype=float64)], dtype=object)

In [230]:
b4

array([array([], dtype=float64), array([157, 218, 414], dtype=uint16),
       array([281, 370, 441], dtype=uint16), ...,
       array([170], dtype=uint8), array([], dtype=float64),
       array([], dtype=float64)], dtype=object)

In [337]:
events_in_this_frame = a4[1]
events_in_this_frame.shape
#a1[events_in_this_frame][0].shape

(1, 1)

In [338]:
events_in_this_frame = b4[1]
events_in_this_frame.shape
#b1[events_in_this_frame][0].shape

(3,)

In [23]:
i = 0
size = 0
for event in a4:
    size += event.size
    if event.size == 0:
        i += 1

In [20]:
size

49213